In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Normalization, CategoryEncoding, StringLookup, IntegerLookup
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
###  Prepare Data (Image + Tabular)
# Load Fashion MNIST data
(X_images, y), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
X_images = X_images[..., np.newaxis] / 255.0  # Normalize

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Generate synthetic tabular data
num_samples = len(X_images)
np.random.seed(42)
prices = np.random.uniform(10, 100, size=num_samples)
ratings = np.random.uniform(1, 5, size=num_samples)
categories = np.random.choice(['A', 'B', 'C'], size=num_samples)

In [4]:
# Create tabular DataFrame
df = pd.DataFrame({'price': prices, 'rating': ratings, 'category': categories, 'label': y})

In [5]:
# Encode categorical feature
df['category_encoded'] = df['category'].map({'A': 0, 'B': 1, 'C': 2})

In [6]:
# Split into train and test
X_tabular = df[['price', 'rating', 'category_encoded']].values
y_labels = df['label'].values
X_tab_train, X_tab_test, X_img_train, X_img_test, y_train, y_test = train_test_split(
    X_tabular, X_images, y_labels, test_size=0.2, random_state=42
)

In [7]:
# Normalize tabular features
scaler = StandardScaler()
X_tab_train_scaled = scaler.fit_transform(X_tab_train)
X_tab_test_scaled = scaler.transform(X_tab_test)

In [8]:
###  Resize Images & Build Image Branch
# Resize 28x28x1 grayscale images to 32x32x3 RGB
X_img_train_resized = tf.image.resize(X_img_train, [32, 32])
X_img_train_rgb = tf.repeat(X_img_train_resized, repeats=3, axis=-1)

X_img_test_resized = tf.image.resize(X_img_test, [32, 32])
X_img_test_rgb = tf.repeat(X_img_test_resized, repeats=3, axis=-1)

In [9]:
# Image input
image_input = Input(shape=(32, 32, 3), name="image_input")


In [10]:
# Pretrained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False
x = base_model(image_input)
x = GlobalAveragePooling2D()(x)

<ipython-input-10-2e35407a9436>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
### Tabular input
tabular_input = Input(shape=(3,), name="tabular_input")
t = Dense(64, activation='relu')(tabular_input)
t = Dense(32, activation='relu')(t)

In [12]:
### Combine both branches
combined = Concatenate()([x, t])
z = Dense(64, activation='relu')(combined)
z = Dense(32, activation='relu')(z)
output = Dense(10, activation='softmax')(z)

In [13]:
# Final model
model = Model(inputs=[image_input, tabular_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
### Training
history = model.fit(
    [X_img_train_rgb, X_tab_train_scaled], y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=32
)

In [ ]:
###  Evaluate
model.evaluate([X_img_test_rgb, X_tab_test_scaled], y_test)

In [ ]:
### Saving the Model
model.save("models/hybrid_model.h5")